<a href="https://colab.research.google.com/github/artms-18/tensorflow_fundamentals/blob/main/Intro_to_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to NLP Fundamentals in Tensorflow

NLP has the goal of deriving information out of natural language (could be sequences text or speech).

Another common term for NLP problems is sequence to sequence problems (seq2seq)

## Check for GPU

In [1]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-b5a905ab-c5e4-aaf1-1d27-a74832ed8745)


## Get helper functions

In [2]:
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

# Import a series of helper function for the notebook

from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

--2021-05-19 12:11:34--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: ‘helper_functions.py’

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2021-05-19 12:11:34 (109 MB/s) - ‘helper_functions.py’ saved [10246/10246]



## Get a text dataaset

The dataset we're going to be using is Kaggle's introduction to NLP dataset

In [3]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data("nlp_getting_started.zip")

--2021-05-19 12:11:36--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.217.164.144, 142.250.73.208, 142.250.73.240, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.217.164.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: ‘nlp_getting_started.zip’

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.004s  

2021-05-19 12:11:36 (136 MB/s) - ‘nlp_getting_started.zip’ saved [607343/607343]



## Visualizing a text dataset

To visualie our text samples, we first have to read them in, one way to do so would be to use Python: 

Another way to do this is to use pandas.

(If text dataset is very large, cannot ise pandas (will tkaes of to much RAM).



In [4]:
import pandas as pd

train_df = pd.read_csv("train.csv")
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [5]:
train_df['text'][0]

'Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all'

In [6]:
# Shuffle training dataframe

train_df_shuffled = train_df.sample(frac = 1, random_state = 42)
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [7]:
# What does the test dataframe look like?

test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [8]:
# How many examples of each class?
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [9]:
# How many total samples?
len(train_df), len(test_df)

(7613, 3263)

In [10]:
# Let's visualize some random training examples

import random

random_index = random.randint(0, len(train_df)-5) # create random indexes not higher than the total number of samples

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
  _, text, target = row #gets rid of index
  print(f"Target: {target}", "(real disaster)" if target > 0 else "(not real disaster")
  print(f"Text:\n{text}\n")
  print("---\n")

Target: 0 (not real disaster
Text:
New and now: Different (FNaF fanfiction): Trixie_drowned / 2 pagesHi my awesome proxies it's ... http://t.co/366NhTg3Tz #wattpad #promo

---

Target: 0 (not real disaster
Text:
Reddit updates content policy promises to quarantine Û÷extremely offensiveÛª communities http://t.co/KmTwA3n1Gf

---

Target: 0 (not real disaster
Text:
Don't think for one second I'm out to drown your memory. Baby you ain't worth the whiskey.

---

Target: 0 (not real disaster
Text:
Crackdown 3 Destruction Restricted to Multiplayer: Crackdown 3 impressed earlier this week with a demonstratio... http://t.co/LMWKjsYCgj

---

Target: 0 (not real disaster
Text:
NHS England announces new plan to meet emergency care targets http://t.co/0x2BIEqXPV

---



### Split data into training and validation sets

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Use train_test_split top split training data into training and validation sets

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(),
                                                                            test_size = 0.1, random_state = 42)

In [13]:
# Check the lengths

len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [14]:
# Check the first 10 samples

train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object), array([0, 

## Converting text into numbers

When dealing with a text problem, one of the first things you'll have to do before you can build a midek is to convert your text to numbers.

There are a few ways to do this, namely:
* Tokenization - direct mapping of  token( a toke could be a word or a character) to number
* embedding - create a matrix of feature vectors for each token the size of the feature vector can be defined and this embedding can be learned

### Text vectorization (tokenization)

In [15]:
train_sentences[:5]

array(['@mogacola @zamtriossu i screamed after hitting tweet',
       'Imagine getting flattened by Kurt Zouma',
       '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
       "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
       'Somehow find you and I collide http://t.co/Ee8RpOahPk'],
      dtype=object)

In [16]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

# Use the default TextVectorization parameters

text_vectorizer = TextVectorization(max_tokens = None, # how many words in the vocab (automatically add <OOV>)
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace", 
                                    ngrams = None,
                                    output_mode = "int", #how to map tokens to numbers
                                    output_sequence_length = None, #pads each sequence to the logest sequence
                                    pad_to_max_tokens = True)

In [17]:
# Find the average number of tokens (words) in the training tweets

round(sum(len(i.split()) for i in train_sentences)) / len(train_sentences)

14.901036345059115

In [18]:
# Setup text vectorization variables

max_vocab_length = 10000 #max num of words to have in our vocabulary
max_length = 15 # max length our seuqences will be (how many words in a tweet, amodel would see)

text_vectorizer = TextVectorization(max_tokens = max_vocab_length, output_mode= 'int', output_sequence_length = max_length)

In [19]:
#fit the text vectorier to the training text

text_vectorizer.adapt(train_sentences)

In [20]:
# Create a sample sentence and tokenize it

sample_sentence = "Theres a flood in my street!"

text_vectorizer([sample_sentence])

<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[264,   3, 232,   4,  13, 698,   0,   0,   0,   0,   0,   0,   0,
          0,   0]])>

In [21]:
# Choose a random sentence from the training dataset and tokenize iot

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 #WorldWatchesFerguson #Florida @GovJayNixon @clairecmc How dare you turn our streets into a war zone -a war against CITIZENS?

Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[6687, 2110,    1,    1,   62, 2503,   12,  916,  103, 1585,   66,
           3,  122,  471,    3]])>

In [22]:
# Get the unique words in the vocabulary

words_in_vocab = text_vectorizer.get_vocabulary() #get all of the unique words in the training data
top_5_words = words_in_vocab[:5] # get the most common words
bottom_5_words = words_in_vocab[-5:] #get the least common words
print(f"Number of words in vocab: {len(words_in_vocab)}")
print(f"5 most comon words: {top_5_words}")

Number of words in vocab: 10000
5 most comon words: ['', '[UNK]', 'the', 'a', 'in']


### Creating an Embedding using an Embedding Layer

To make our embedding, we're going to use tensoflows embedding layer

The parameters we actually care about:
* `input_dim` = the size of our vocabulary
* `output_dim` = the size of the output embedding vector, for example, a value of 100 would mean each token gets represented by a vecotr 100 long
* `input_length` = length of the sequences being passed to the embedding layer

In [23]:
from tensorflow.keras import layers

# in keras, wordks best when numbers are divisible by eight

embedding = layers.Embedding(input_dim = max_vocab_length,
                             output_dim = 128,
                             input_length = max_length
                             )
embedding

In [24]:
# Get a random sentence from the training set

random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\n\nEmbedded version:")

# Embed the random sentence (turn into dense vecotrs of fixed size)
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 Mama I'm still thuggin the world is a war zone my homies is inmate and most of em dead wrong.

Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04891508,  0.00133355,  0.03100068, ...,  0.03198076,
          0.00771584,  0.02631621],
        [ 0.00277073,  0.0330187 , -0.01406398, ...,  0.04356562,
          0.04761452,  0.01171369],
        [ 0.01120976,  0.04351958, -0.03408929, ..., -0.00014192,
         -0.01995852,  0.00440411],
        ...,
        [ 0.02864448, -0.00498967, -0.0388552 , ...,  0.04233483,
          0.03624347, -0.02076697],
        [ 0.04883141,  0.03774257,  0.03691156, ...,  0.00495027,
          0.02485862, -0.03939676],
        [ 0.01949555,  0.00221227, -0.02516289, ...,  0.03136155,
          0.04208573, -0.0192222 ]]], dtype=float32)>

In [25]:
# Chekc out a single token's embedding

sample_embed[0][0], sample_embed[0][0].shape, random_sentence[0]

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04891508,  0.00133355,  0.03100068, -0.03917714, -0.0126339 ,
         0.00683988, -0.03167924, -0.0190185 ,  0.02184613,  0.01456794,
         0.0375105 , -0.03167715,  0.01725764, -0.04822215, -0.03084376,
        -0.01439924, -0.00203819, -0.02878801,  0.01361526,  0.03360495,
        -0.01007072, -0.02598574, -0.03436575,  0.04564662,  0.01442686,
        -0.04291818,  0.04424231, -0.00349885,  0.03078785, -0.00618826,
        -0.02684963,  0.02331573, -0.03868927, -0.00291923,  0.00551846,
         0.02016873, -0.04412545, -0.01880571,  0.04986257,  0.00067301,
        -0.03585313, -0.04664345,  0.02902726, -0.01204104, -0.00801116,
         0.02623982,  0.03951974,  0.02932478, -0.04026133, -0.00529939,
         0.04275315,  0.04648316, -0.01027902,  0.02210024, -0.03229425,
         0.02128896, -0.03329878,  0.00099499, -0.01932708,  0.02794145,
         0.04752203,  0.00350274, -0.0487613 , -0.03392165, -0.04370746,
  

## Modelling a text dataset (running a series of experiments)

Now we're got a wat to turn our text srequences into numbers, it's time to start building a series of modelling experiments, start from a baseline and move on from there 

* Model 0: Naive Bayes (baseline)
* Model 1: Feed-forward neual network (dense model)
* Model 2: GRU model (RNN)
* Model 4: Bidirectional-LSTM
* Model 5: 1d Convolutional Neural Network (CNN)
* Model 6: Tensorflow Hub Pretrained Geature Extractor (using transfer learning for NLP)
* Model 7: Same as model 6 with 10% of training data

How are we going to approach all of these?

Use the standard steps in modelling with tensorflow:

* Create a model
* Build a model
* Compile a model
* Fit the model

### Model 0: Getting a baseline

As with all machine learning modelling experiments it's important to create a baseline model so u got a benchmark for future experiments to build upon

To create our baseline, we'll use SKlearns Multinomial Naive Bayes using the TF-IDF formula to convert our words to numbers

> **Note:** It's common practive to use non-DL algorithms as a baseline because of their speed, then later using DL to improve

In [26]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Create tokenization and modelling pipeline

model_0 = Pipeline([
                    ("tfidf", TfidfVectorizer()),
                    ("clf", MultinomialNB())
])

# Fit the pipeline to the training data

model_0.fit(train_sentences, train_labels)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('clf',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [27]:
# Evaluate our baseline model 

baseline_score = model_0.score(val_sentences, val_labels)
print(f"Our baseline model achieves an accuracy of: {baseline_score*100:.2f}%")

Our baseline model achieves an accuracy of: 79.27%


In [28]:
# Make predictions

baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

### Creating an evaluation function for our model experiments

We could evaluate model's predictin with different methods every time, but this could be cumbersome and could easily be fixed with a function

Let's create one to compare our model's predictions with the truth labels using the following metrics:

* Accuracy
* Precision
* Recall
* F1-score

In [29]:
# Function to evaluate: accuracty, precision, recall, f1-score

from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def calculate_results(y_true, y_pred):
  """
  Calulates model accuracy, precision, recall and f1 score of a binary classification model.
  """
  #Calculate model accuracy
  model_accuracy = accuracy_score(y_true, y_pred) * 100
  #Calculate model precision, recall ,and f01 score using "weighted" average
  model_precision, model_recall, model_f1, _ = precision_recall_fscore_support(y_true, y_pred, average = 'weighted')
  model_results = {"accuracy": model_accuracy,
                   "precision": model_precision,
                   "recall": model_recall,
                   "f1": model_f1}
  return model_results

In [30]:
baseline_results = calculate_results(y_true = val_labels, y_pred = baseline_preds)

baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

### Model 1: A simple dense model

In [31]:
# Create a tensorboard callback (need to create a new one for each model)

from helper_functions import create_tensorboard_callback

# Create a directory to save TensorBoard logs

SAVE_DIR = "model_logs"

In [32]:
from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = tf.string) # inputs are 1-dimensional
x = text_vectorizer(inputs) # turn the input text into numbers
x = embedding(x) # create an embedding of the numberized imputs
x = layers.GlobalAveragePooling1D()(x) # condense the feature vector for each token to one vector
#x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1, activation = "sigmoid")(x) # Create the output layer, want binary output so use sigmoid

model_1 = tf.keras.Model(inputs, outputs, name = "model_1_dense")

In [33]:
model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [34]:
# Compile mode

model_1.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [35]:
# Fit the mode
model_1_history = model_1.fit(x = train_sentences, y = train_labels, epochs = 15, validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(dir_name = SAVE_DIR, experiment_name = "model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20210519-121143
Epoch 1/15
215/215 [==============================] - 7s 18ms/step - loss: 0.6553 - accuracy: 0.6289 - val_loss: 0.5369 - val_accuracy: 0.7559
Epoch 2/15
215/215 [==============================] - 3s 14ms/step - loss: 0.4602 - accuracy: 0.8094 - val_loss: 0.4733 - val_accuracy: 0.7835
Epoch 3/15
215/215 [==============================] - 3s 13ms/step - loss: 0.3540 - accuracy: 0.8571 - val_loss: 0.4591 - val_accuracy: 0.7940
Epoch 4/15
215/215 [==============================] - 3s 13ms/step - loss: 0.2829 - accuracy: 0.8922 - val_loss: 0.4651 - val_accuracy: 0.7900
Epoch 5/15
215/215 [==============================] - 3s 13ms/step - loss: 0.2294 - accuracy: 0.9152 - val_loss: 0.4760 - val_accuracy: 0.7874
Epoch 6/15
215/215 [==============================] - 3s 13ms/step - loss: 0.1955 - accuracy: 0.9314 - val_loss: 0.4983 - val_accuracy: 0.7887
Epoch 7/15
215/215 [==============================] - 3s 14ms/step -

In [36]:
# Check the results

model_1.evaluate(val_sentences, val_labels)

24/24 [==============================] - 0s 3ms/step - loss: 0.7620 - accuracy: 0.7677


[0.7620409727096558, 0.7677165269851685]

In [37]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs.shape

(762, 1)

In [38]:
# look at a single prediction

model_1_pred_probs[0] # anything below 0.5 = disaster, abouve = non-=disaster

array([0.56189114], dtype=float32)

In [39]:
# look at the first 10 predictions

model_1_pred_probs[:10]

array([[0.56189114],
       [0.60060275],
       [0.99926096],
       [0.0745422 ],
       [0.00142469],
       [0.9867089 ],
       [0.8315361 ],
       [0.99997866],
       [0.9999453 ],
       [0.6284906 ]], dtype=float32)

In [40]:
# Convert model prediction probablilities to label format

model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

model_1_preds[:20]

<tf.Tensor: shape=(20,), dtype=float32, numpy=
array([1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1.], dtype=float32)>

In [41]:
# calculate our model_1 results

model_1_results = calculate_results(y_true = val_labels, y_pred = model_1_preds)

model_1_results

{'accuracy': 76.77165354330708,
 'f1': 0.7662770891654436,
 'precision': 0.7681410880728078,
 'recall': 0.7677165354330708}

In [42]:
baseline_results

{'accuracy': 79.26509186351706,
 'f1': 0.7862189758049549,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706}

In [43]:
import numpy as np

np.array(list(model_1_results.values())) > np.array(list(baseline_results.values()))

array([False, False, False, False])

# Visualizing learned embeddings


In [44]:
# get the vocab from the text vecotrization layer

words_in_vocab = text_vectorizer.get_vocabulary()
len(words_in_vocab), words_in_vocab[:10]

(10000, ['', '[UNK]', 'the', 'a', 'in', 'to', 'of', 'and', 'i', 'is'])

In [45]:
# Model 1 summary

model_1.summary()

Model: "model_1_dense"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
global_average_pooling1d (Gl (None, 128)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,280,129
Trainable params: 1,280,129
Non-trainable params: 0
_________________________________________________________________


In [46]:
# Get the wieght matrix of embedding layer
# (these are the numerical representations of each token in our training data, which have been learned for 5 epocks)

embed_weights = model_1.get_layer("embedding").get_weights()[0]

print(embed_weights.shape) # same size as vocab size and embedding dim (output dim of our embedding layer)

(10000, 128)


In [47]:
embed_weights

array([[ 0.00947826,  0.02020008, -0.05642007, ..., -0.05271576,
         0.00851127,  0.01863865],
       [ 0.05275297,  0.043735  ,  0.03914997, ...,  0.00748001,
         0.02306955, -0.04305607],
       [ 0.00529411, -0.05163547,  0.01382198, ..., -0.0560341 ,
        -0.01192013,  0.00584223],
       ...,
       [ 0.0250824 ,  0.02571781, -0.00052166, ..., -0.03032793,
         0.0107439 ,  0.02766773],
       [-0.06780215, -0.01158283, -0.05338255, ..., -0.06787065,
         0.0688256 ,  0.0052491 ],
       [-0.14988258, -0.12228397, -0.14327033, ..., -0.15910704,
         0.15513563,  0.16541485]], dtype=float32)

In [48]:
# Create embedding files

import io

out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  #if index == 0:
   # continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [49]:
# Download files from Colab to upload to projector

try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Recurrent Neural Networks (RNN's)

RNN's are usefull for sequence data.

The premise of a recurrent neural network is to use the representation of a previous input to aid the respresentation of a later input.

### Model 2: LSTM

LSTM = ling short term memory (one of the most popular LSTM cells)

Our structure of an RNN typically looks like this:

'''
Input (text) -> Tokenize -> EMbedding -> Layers (RNN's/dense) -> Output (label probability)
'''

In [50]:
# Create an LSTM model

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
print(x.shape)
#x = layers.LSTM(64, return_sequences = True)(x)
print(x.shape)
x = layers.LSTM(64)(x) # when you're stacking RNN cells together, you need to return_sequences = true
print(x.shape)
#x = layers.Dense(64, activation = 'relu')(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)

model_2 = tf.keras.Model(inputs, outputs, name = "model_2_LSTM")

(None, 15, 128)
(None, 15, 128)
(None, 64)


In [51]:
# Compile the model

model_2.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ["accuracy"])

In [52]:
model_2_history = model_2.fit(train_sentences, train_labels, epochs = 5, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_2_LSTM")])

Saving TensorBoard log files to: model_logs/model_2_LSTM/20210519-121232
Epoch 1/5
215/215 [==============================] - 34s 22ms/step - loss: 0.2051 - accuracy: 0.9505 - val_loss: 0.9209 - val_accuracy: 0.7717
Epoch 2/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0680 - accuracy: 0.9749 - val_loss: 0.9988 - val_accuracy: 0.7717
Epoch 3/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0539 - accuracy: 0.9784 - val_loss: 0.9518 - val_accuracy: 0.7664
Epoch 4/5
215/215 [==============================] - 3s 16ms/step - loss: 0.0560 - accuracy: 0.9756 - val_loss: 1.2406 - val_accuracy: 0.7677
Epoch 5/5
215/215 [==============================] - 3s 15ms/step - loss: 0.0474 - accuracy: 0.9798 - val_loss: 1.2484 - val_accuracy: 0.7638


In [53]:
# Make predictions with LSTM model

model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]


array([[1.5445837e-01],
       [5.1043099e-01],
       [9.9955696e-01],
       [1.7052971e-01],
       [9.6745964e-05],
       [9.9979120e-01],
       [9.3679976e-01],
       [9.9993873e-01],
       [9.9992406e-01],
       [9.9960285e-01]], dtype=float32)

In [54]:
# Convert model 2 pred probs to labels

model_2_preds = tf.squeeze(tf.round(model_2_pred_probs))
model_2_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [55]:
model_2_results = calculate_results(y_true = val_labels, y_pred = model_2_preds)
model_2_results

{'accuracy': 76.37795275590551,
 'f1': 0.7629812681164898,
 'precision': 0.7634711836373635,
 'recall': 0.7637795275590551}

### Model 3: GRU

Another poppular and effective RNN component is the GRU or gated recurrent unit.

The GRU cell has similar features to an LSTM cell but has less parameters.

In [56]:
# Build an RNN using the GRU cell
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
#x = layers.LSTM(64, return_sequences = True)(x)
#print(x.shape)
#x = layers.GRU(64)(x) # make sure return seuqences DOES NO equal true OR add a global average pooling layer instead
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)


model_3 = tf.keras.Model(inputs, outputs)

In [57]:
 model_3.summary(
     
 )

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
gru (GRU)                    (None, 64)                37248     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 65        
Total params: 1,317,313
Trainable params: 1,317,313
Non-trainable params: 0
_________________________________________________________________


In [58]:
# COmpile the model

model_3.compile(loss = "binary_crossentropy", optimizer= tf.keras.optimizers.Adam(), metrics = ["accuracy"])


In [59]:
# Fit

model_3_history = model_3.fit(train_sentences,
                              train_labels, epochs = 10,
                              validation_data = (val_sentences, val_labels),
                              callbacks = [create_tensorboard_callback(SAVE_DIR, "model_3_GRU")])

Saving TensorBoard log files to: model_logs/model_3_GRU/20210519-121321
Epoch 1/10
215/215 [==============================] - 6s 20ms/step - loss: 0.2465 - accuracy: 0.8900 - val_loss: 0.8497 - val_accuracy: 0.7638
Epoch 2/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0548 - accuracy: 0.9769 - val_loss: 0.9947 - val_accuracy: 0.7585
Epoch 3/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0505 - accuracy: 0.9775 - val_loss: 1.0646 - val_accuracy: 0.7690
Epoch 4/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0438 - accuracy: 0.9830 - val_loss: 0.9743 - val_accuracy: 0.7625
Epoch 5/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0450 - accuracy: 0.9837 - val_loss: 1.1633 - val_accuracy: 0.7546
Epoch 6/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0411 - accuracy: 0.9798 - val_loss: 1.0543 - val_accuracy: 0.7598
Epoch 7/10
215/215 [==============================] - 3s 15ms/step - l

In [60]:
# Make some predictions with GRu model

model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred_probs[:10]

array([[3.20180407e-04],
       [8.30139697e-01],
       [9.99910474e-01],
       [1.04569845e-01],
       [6.91069363e-05],
       [9.99902129e-01],
       [9.79381144e-01],
       [9.99981046e-01],
       [9.99977231e-01],
       [9.99872446e-01]], dtype=float32)

In [61]:
model_3_preds = tf.squeeze(tf.round(model_3_pred_probs))
model_3_preds[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [62]:
# Calculate model 3 results

model_3_results = calculate_results(y_true = val_labels, y_pred = model_3_preds)
model_3_results

{'accuracy': 76.37795275590551,
 'f1': 0.7626922180365151,
 'precision': 0.76369847151459,
 'recall': 0.7637795275590551}

### Model 4: Bidirectional RNN

ormal RNN's go from left to right (just like you'd read an English sentence) however, a bidiractional RNN goes from right to left as well as left to right.

In [63]:
# Build a dirirectional RNN in Tensorflow

from tensorflow.keras import layers

inputs = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
#x = layers.Bidirectional(layers.LSTM(64, return_sequences = True))(x)
#print(x.shape)
x = layers.Bidirectional(layers.GRU(64))(x)
print(x.shape)
outputs = layers.Dense(1, activation = 'sigmoid')(x)
model_4 = tf.keras.Model(inputs, outputs, name = "model_4_bidirectional")

(None, 128)


In [64]:
# Get a summary

model_4.summary()

Model: "model_4_bidirectional"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
bidirectional (Bidirectional (None, 128)               74496     
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 1,354,625
Trainable params: 1,354,625
Non-trainable params: 0
_________________________________________________________________


In [65]:
# Compile model

model_4.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

In [66]:
model_4_history = model_4.fit(train_sentences, train_labels, epochs = 5, validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR,
                                                                                                                                                                 "model_4_bidirectional")])

Saving TensorBoard log files to: model_logs/model_4_bidirectional/20210519-121357
Epoch 1/5
215/215 [==============================] - 7s 23ms/step - loss: 0.1852 - accuracy: 0.9573 - val_loss: 0.9284 - val_accuracy: 0.7612
Epoch 2/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0433 - accuracy: 0.9818 - val_loss: 1.2422 - val_accuracy: 0.7572
Epoch 3/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0322 - accuracy: 0.9873 - val_loss: 1.3169 - val_accuracy: 0.7467
Epoch 4/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0351 - accuracy: 0.9842 - val_loss: 1.4349 - val_accuracy: 0.7585
Epoch 5/5
215/215 [==============================] - 4s 17ms/step - loss: 0.0397 - accuracy: 0.9820 - val_loss: 1.6074 - val_accuracy: 0.7598


In [67]:
# Make predictions with out bidiractional ,odel
 
model_4_pred_probs = model_4.predict(val_sentences)
model_4_pred_probs[:10]

array([[3.0532331e-04],
       [7.9093736e-01],
       [9.9994552e-01],
       [1.4294478e-01],
       [1.1088872e-05],
       [9.9972945e-01],
       [9.9756324e-01],
       [9.9999487e-01],
       [9.9997950e-01],
       [9.9977976e-01]], dtype=float32)

In [68]:
# Convert

model_4_pred = tf.squeeze(tf.round(model_4_pred_probs))
model_4_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 1., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [69]:
# Calculate the results of our bidiractional model

model_4_results = calculate_results(y_true = val_labels, y_pred = model_4_pred )
model_4_results

{'accuracy': 75.98425196850394,
 'f1': 0.7583542786286791,
 'precision': 0.7601337255372377,
 'recall': 0.7598425196850394}

### Model 5: Conv1D 

We've used CNNs for images but images are typically 2D (hegith x width)...however, our text data is 1d.

The typical structure of a Conv1D model for sequen ces (in our case, text):

```
Inputs(test -> Tokenization -> Embedding -> Cpmv1D -> Pool -> Outputs (class probs)
```

In [70]:
from tensorflow.keras import layers

input = layers.Input(shape = (1,), dtype = "string")
x = text_vectorizer(input)
x = embedding(x)
x = layers.Conv1D(32, 6, activation = 'relu')(x) #kernal size is reffered to as n-gram side (strides rn is 1, but can change to hop) #padding = 'valid' (some words mised) output is smaller 'same' = smae size
x = layers.GlobalMaxPooling1D()(x)
output = layers.Dense(1, activation = "sigmoid")(x)

model_5 = tf.keras.Model(input, output, name = "model_5_conv")


In [71]:
model_5.summary()

Model: "model_5_conv"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_5 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d (Conv1D)              (None, 10, 32)            24608     
_________________________________________________________________
global_max_pooling1d (Global (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,304,641
Trainable params: 1,304,641
Non-trainable params: 0
____________________________________________

In [72]:
# Compiling the model

model_5.compile(loss = 'binary_crossentropy', optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])



In [73]:
# Fitting the model

model_5_history = model_5.fit(train_sentences, train_labels, epochs = 10, 
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_5_conv")])

Saving TensorBoard log files to: model_logs/model_5_conv/20210519-121421
Epoch 1/10
215/215 [==============================] - 6s 19ms/step - loss: 0.1635 - accuracy: 0.9564 - val_loss: 1.0051 - val_accuracy: 0.7533
Epoch 2/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0568 - accuracy: 0.9792 - val_loss: 1.1363 - val_accuracy: 0.7546
Epoch 3/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0489 - accuracy: 0.9782 - val_loss: 1.2259 - val_accuracy: 0.7546
Epoch 4/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0423 - accuracy: 0.9810 - val_loss: 1.3045 - val_accuracy: 0.7546
Epoch 5/10
215/215 [==============================] - 3s 15ms/step - loss: 0.0401 - accuracy: 0.9822 - val_loss: 1.3716 - val_accuracy: 0.7480
Epoch 6/10
215/215 [==============================] - 3s 14ms/step - loss: 0.0388 - accuracy: 0.9828 - val_loss: 1.3852 - val_accuracy: 0.7467
Epoch 7/10
215/215 [==============================] - 3s 14ms/step - 

In [74]:
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64, kernel_size = 5, activation = "relu")(x)
x = layers.GlobalMaxPool1D()(x)
#x = layers.Dense(64, activation = "relu")(x)
outputs = layers.Dense(1, activation = "sigmoid")(x)
model_6 = tf.keras.Model(inputs, outputs, name = "model_6_Conv1D")

model_6.compile(loss = "binary_crossentropy", optimizer = tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_6.summary()

Model: "model_6_Conv1D"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 1)]               0         
_________________________________________________________________
text_vectorization_1 (TextVe (None, 15)                0         
_________________________________________________________________
embedding (Embedding)        (None, 15, 128)           1280000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 11, 64)            41024     
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 1,321,089
Trainable params: 1,321,089
Non-trainable params: 0
__________________________________________

In [75]:
# Fit the model

model_6_history = model_6.fit(train_sentences, train_labels, epochs = 5,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "Conv1D")])

Saving TensorBoard log files to: model_logs/Conv1D/20210519-121455
Epoch 1/5
215/215 [==============================] - 5s 18ms/step - loss: 0.1195 - accuracy: 0.9712 - val_loss: 1.1440 - val_accuracy: 0.7533
Epoch 2/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0436 - accuracy: 0.9827 - val_loss: 1.2477 - val_accuracy: 0.7415
Epoch 3/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0396 - accuracy: 0.9828 - val_loss: 1.3661 - val_accuracy: 0.7428
Epoch 4/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0416 - accuracy: 0.9798 - val_loss: 1.3925 - val_accuracy: 0.7454
Epoch 5/5
215/215 [==============================] - 3s 14ms/step - loss: 0.0380 - accuracy: 0.9821 - val_loss: 1.4530 - val_accuracy: 0.7428


In [76]:
# Make some prediction with our Conv1D model

model_6_pred_probs = model_6.predict(val_sentences)
model_6_pred = tf.squeeze(tf.round(model_6_pred_probs))
model_6_pred[:10]

<tf.Tensor: shape=(10,), dtype=float32, numpy=array([0., 0., 1., 0., 0., 1., 1., 1., 1., 1.], dtype=float32)>

In [77]:
model_6_results = calculate_results(y_true = val_labels, y_pred = model_6_pred)
model_6_results

{'accuracy': 74.2782152230971,
 'f1': 0.7407315702114267,
 'precision': 0.7432964552031615,
 'recall': 0.7427821522309711}

## Model6: Tensorflow Hub Pretrained Sentence Encoder

Using transfer learning for NLP

In [78]:
import tensorflow_hub as hub

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")
embed_samples = embed([sample_sentence, "When you call the use on a sentence, it utrns it into numbers"])

print(embed_samples[0][:50])

tf.Tensor(
[-0.01157027  0.02485911  0.02878048 -0.012715    0.03971538  0.08827761
  0.02680985  0.05589839 -0.01068729 -0.00597292  0.00639323 -0.0181952
  0.00030814  0.09105889  0.05874645 -0.03180628  0.01512474 -0.05162929
  0.00991367 -0.06865346 -0.04209306  0.02678981  0.03011008  0.00321069
 -0.00337971 -0.04787357  0.02266719 -0.00985925 -0.04063613 -0.01292093
 -0.04666385  0.056303   -0.03949255  0.00517688  0.02495828 -0.07014441
  0.02871509  0.04947684 -0.00633978 -0.08960193  0.02807117 -0.00808362
 -0.01360601  0.0599865  -0.10361787 -0.05195374  0.00232955 -0.0233253
 -0.03758106  0.0332773 ], shape=(50,), dtype=float32)


In [80]:
# transforms THE WHOLE sentence into a 512 dimensional vector

In [81]:
# Create Keras Layer using the USE pretrained layer from tensorflow hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape = [],
                                        dtype = tf.string,
                                        trainable = False,
                                        name = "USE")

In [97]:
# Create model using the sequenctial API

model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64, activation = "relu"),
    layers.Dense(1, activation = 'sigmoid', name = 'output_layer')
], name = "model_7_USE")

# Compile

model_7.compile(loss = 'binary_crossentropy', optimizer = 
                tf.keras.optimizers.Adam(), metrics = ['accuracy'])

model_7.summary()

Model: "model_7_USE"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
USE (KerasLayer)             (None, 512)               256797824 
_________________________________________________________________
dense_16 (Dense)             (None, 64)                32832     
_________________________________________________________________
output_layer (Dense)         (None, 1)                 65        
Total params: 256,830,721
Trainable params: 32,897
Non-trainable params: 256,797,824
_________________________________________________________________


In [98]:
# Train a classifier on top of USE pretrained embeddings

model_7_history = model_7.fit(train_sentences, train_labels, epochs = 15,
                              validation_data = (val_sentences, val_labels), callbacks = [create_tensorboard_callback(SAVE_DIR, "model_7_use")])

Saving TensorBoard log files to: model_logs/model_7_use/20210519-124748
Epoch 1/15
215/215 [==============================] - 4s 16ms/step - loss: 0.5813 - accuracy: 0.7352 - val_loss: 0.4496 - val_accuracy: 0.7979
Epoch 2/15
215/215 [==============================] - 2s 10ms/step - loss: 0.4161 - accuracy: 0.8144 - val_loss: 0.4367 - val_accuracy: 0.8123
Epoch 3/15
215/215 [==============================] - 2s 10ms/step - loss: 0.4007 - accuracy: 0.8173 - val_loss: 0.4320 - val_accuracy: 0.8163
Epoch 4/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3902 - accuracy: 0.8309 - val_loss: 0.4288 - val_accuracy: 0.8123
Epoch 5/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3794 - accuracy: 0.8331 - val_loss: 0.4247 - val_accuracy: 0.8215
Epoch 6/15
215/215 [==============================] - 2s 10ms/step - loss: 0.3758 - accuracy: 0.8360 - val_loss: 0.4262 - val_accuracy: 0.8228
Epoch 7/15
215/215 [==============================] - 2s 10ms/step - l

In [99]:
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
results = calculate_results(val_labels, model_7_preds)
results

{'accuracy': 82.80839895013123,
 'f1': 0.8271790179159411,
 'precision': 0.8290723185352373,
 'recall': 0.8280839895013123}